In [9]:
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch

In [2]:
from adult_read import *

In [3]:
class LoadData(Dataset):
    def __init__(self, df, pred_var):
        self.y = df[pred_var].values
        self.x = df.drop(pred_var, axis = 1).values
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.y.shape[0]

In [15]:
################## MODEL SETTING ########################
BATCH_SIZE = 128
NUM_WORKERS = 4
RANDOM_SEED = 123
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [17]:
client1_dataset = LoadData(client1, 'salary')
client1_loader = DataLoader(dataset = client1_dataset,
                            batch_size = BATCH_SIZE,
                            num_workers = NUM_WORKERS)

client2_dataset = LoadData(client2, 'salary')
client2_loader = DataLoader(dataset = client2_dataset,
                            batch_size = BATCH_SIZE,
                            num_workers = NUM_WORKERS)

test_dataset = LoadData(test, 'salary')
test_loader = DataLoader(dataset = test_dataset,
                            batch_size = BATCH_SIZE,
                            num_workers = NUM_WORKERS)

In [18]:
torch.manual_seed(0)

num_epochs = 2
for epoch in range(num_epochs):

    for batch_idx, (x, y) in enumerate(client1_loader):
        
        print('Epoch:', epoch+1, end='')
        print(' | Batch index:', batch_idx, end='')
        print(' | Batch size:', y.size()[0])
        
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        
        print('break minibatch for-loop')
        break

Epoch: 1 | Batch index: 0 | Batch size: 128
break minibatch for-loop
Epoch: 2 | Batch index: 0 | Batch size: 128
break minibatch for-loop
